# Data Holder - Key Generation, Encryption & Decryption

This notebook manages the secret key and performs encryption/decryption operations.

## Imports

In [1]:
import tenseal as ts
import utils
import json

------------------------------------------------
### 1. Generating Keys:

In [2]:
# Initialize CKKS encryption context for float values (radiation)
# poly_modulus_degree=8192: security level (higher = more secure, slower)
# coeff_mod_bit_sizes: determines how many operations can be performed
ckks_encryption_context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60],
)

# Initialize BFV encryption context for integer values (enemy counts)
# poly_modulus_degree=16384: higher value allows more additions without errors
# plain_modulus=786433: must be prime and compatible with poly_modulus_degree
bfv_encryption_context = ts.context(
    ts.SCHEME_TYPE.BFV,
    poly_modulus_degree=16384,
    plain_modulus=786433,
)

In [3]:
# Generate key for CKKS homomorphic operations
ckks_encryption_context.generate_galois_keys()
ckks_encryption_context.global_scale = 2**40

# Generate key for BFV homomorphic operations
bfv_encryption_context.generate_galois_keys()

------------------------------------------------------------------------------------

## Save Secret and Public Keys

Export both keys to files for later use.

In [4]:
# Serialize CKKS context with secret key
serialized_ckks_secret_context = ckks_encryption_context.serialize(save_secret_key=True)
utils.write_data("keys/ckks_secret.txt", serialized_ckks_secret_context)

# Serialize BFV context with secret key
serialized_bfv_secret_context = bfv_encryption_context.serialize(save_secret_key=True)
utils.write_data("keys/bfv_secret.txt", serialized_bfv_secret_context)

print("✅ CKKS secret key saved to keys/ckks_secret.txt")
print("✅ BFV secret key saved to keys/bfv_secret.txt")

✅ CKKS secret key saved to keys/ckks_secret.txt
✅ BFV secret key saved to keys/bfv_secret.txt


In [5]:
# Make CKKS context public for Data Analyzer
ckks_encryption_context.make_context_public()
serialized_ckks_public_context = ckks_encryption_context.serialize()
utils.write_data("keys/ckks_public.txt", serialized_ckks_public_context)

# Make BFV context public for Data Analyzer
bfv_encryption_context.make_context_public()
serialized_bfv_public_context = bfv_encryption_context.serialize()
utils.write_data("keys/bfv_public.txt", serialized_bfv_public_context)

print("✅ CKKS public key saved to keys/ckks_public.txt")
print("✅ BFV public key saved to keys/bfv_public.txt")

✅ CKKS public key saved to keys/ckks_public.txt
✅ BFV public key saved to keys/bfv_public.txt


------------------------------------------

## Encryption

In [6]:
# Reload CKKS secret context from file
ckks_secret_context_bytes = utils.read_data("keys/ckks_secret.txt")
ckks_secret_context = ts.context_from(ckks_secret_context_bytes)

# Reload BFV secret context from file
bfv_secret_context_bytes = utils.read_data("keys/bfv_secret.txt")
bfv_secret_context = ts.context_from(bfv_secret_context_bytes)

In [7]:
# Load plaintext radiation measurements (floats for CKKS)
with open("data/raw_data_ckks.json", "r") as json_file:
    plaintext_radiation_measurements = json.load(json_file)

# Load plaintext enemy counts (integers for BFV)
with open("data/raw_data_bfv.json", "r") as json_file:
    plaintext_enemy_counts = json.load(json_file)

print(f"Loaded {len(plaintext_radiation_measurements)} radiation measurements (CKKS)")
print(f"Loaded {len(plaintext_enemy_counts)} enemy counts (BFV)")

Loaded 1500 radiation measurements (CKKS)
Loaded 1500 enemy counts (BFV)


In [8]:
# Encrypt radiation measurements using CKKS scheme (for floats)
encrypted_radiation_vector = ts.ckks_vector(
    ckks_secret_context, 
    plaintext_radiation_measurements
)

# Encrypt enemy counts using BFV scheme (for integers)
encrypted_enemy_vector = ts.bfv_vector(
    bfv_secret_context,
    plaintext_enemy_counts
)

print("✅ Radiation data encrypted with CKKS")
print("✅ Enemy count data encrypted with BFV")

✅ Radiation data encrypted with CKKS
✅ Enemy count data encrypted with BFV


In [9]:
# Save encrypted radiation data (CKKS)
serialized_encrypted_radiation = encrypted_radiation_vector.serialize()
utils.write_data("outputs/radiation_encrypted.txt", serialized_encrypted_radiation)

# Save encrypted enemy counts (BFV)
serialized_encrypted_enemy = encrypted_enemy_vector.serialize()
utils.write_data("outputs/enemy_encrypted.txt", serialized_encrypted_enemy)

print("✅ Encrypted radiation data saved to outputs/radiation_encrypted.txt")
print("✅ Encrypted enemy data saved to outputs/enemy_encrypted.txt")

✅ Encrypted radiation data saved to outputs/radiation_encrypted.txt
✅ Encrypted enemy data saved to outputs/enemy_encrypted.txt


------------------------------------------

## Encrypt Calculation Parameters

Encrypt weights and thresholds to hide calculation logic from the Analyzer.

In [10]:
# Load number of measurements for divisor calculation
with open("data/expected_values.json", "r") as json_file:
    expected_data = json.load(json_file)
    total_number_of_measurements = expected_data["number_of_measurements"]

# Encrypt strategic weights for threat calculation
# These define the calculation logic that the Analyzer should not see
encrypted_weight_radiation = ts.ckks_vector(ckks_secret_context, [0.7])
encrypted_weight_enemy = ts.ckks_vector(ckks_secret_context, [0.3])

# Encrypt divisor for average calculation (hides number of measurements)
encrypted_divisor = ts.ckks_vector(ckks_secret_context, [1.0 / total_number_of_measurements])

# Encrypt thresholds for high-risk zone detection
encrypted_threshold_radiation = ts.ckks_vector(ckks_secret_context, [50.0])
encrypted_threshold_enemies = ts.ckks_vector(ckks_secret_context, [10.0])

# Encrypt safe route constant
encrypted_safe_route_constant = ts.ckks_vector(ckks_secret_context, [100.0])

print("✅ Encrypted calculation parameters (weights, thresholds, divisor)")

✅ Encrypted calculation parameters (weights, thresholds, divisor)


In [11]:
# Save encrypted parameters for the Analyzer
utils.write_data("outputs/weight_radiation_encrypted.txt", encrypted_weight_radiation.serialize())
utils.write_data("outputs/weight_enemy_encrypted.txt", encrypted_weight_enemy.serialize())
utils.write_data("outputs/divisor_encrypted.txt", encrypted_divisor.serialize())
utils.write_data("outputs/threshold_radiation_encrypted.txt", encrypted_threshold_radiation.serialize())
utils.write_data("outputs/threshold_enemies_encrypted.txt", encrypted_threshold_enemies.serialize())
utils.write_data("outputs/safe_route_constant_encrypted.txt", encrypted_safe_route_constant.serialize())

print("✅ Saved all encrypted parameters to outputs/")
print("   - weight_radiation_encrypted.txt (0.7)")
print("   - weight_enemy_encrypted.txt (0.3)")
print("   - divisor_encrypted.txt (1/n)")
print("   - threshold_radiation_encrypted.txt (50.0 mSv)")
print("   - threshold_enemies_encrypted.txt (10.0 units)")
print("   - safe_route_constant_encrypted.txt (100.0)")
print("   → Analyzer cannot see plaintext calculation logic")

✅ Saved all encrypted parameters to outputs/
   - weight_radiation_encrypted.txt (0.7)
   - weight_enemy_encrypted.txt (0.3)
   - divisor_encrypted.txt (1/n)
   - threshold_radiation_encrypted.txt (50.0 mSv)
   - threshold_enemies_encrypted.txt (10.0 units)
   - safe_route_constant_encrypted.txt (100.0)
   → Analyzer cannot see plaintext calculation logic


------------------------------------------

## Decryption

In [12]:
# Load encrypted radiation average (CKKS)
encrypted_average_bytes = utils.read_data("outputs/radiation_average_encrypted.txt")
encrypted_average_vector = ts.lazy_ckks_vector_from(encrypted_average_bytes)
encrypted_average_vector.link_context(ckks_secret_context)

# Load encrypted enemy sum (BFV)
encrypted_sum_bytes = utils.read_data("outputs/enemy_sum_encrypted.txt")
encrypted_sum_vector = ts.lazy_bfv_vector_from(encrypted_sum_bytes)
encrypted_sum_vector.link_context(bfv_secret_context)

# Load additional CKKS results
encrypted_radiation_score_bytes = utils.read_data("outputs/radiation_score_encrypted.txt")
encrypted_radiation_score_vector = ts.lazy_ckks_vector_from(encrypted_radiation_score_bytes)
encrypted_radiation_score_vector.link_context(ckks_secret_context)

encrypted_safe_route_bytes = utils.read_data("outputs/safe_route_score_encrypted.txt")
encrypted_safe_route_vector = ts.lazy_ckks_vector_from(encrypted_safe_route_bytes)
encrypted_safe_route_vector.link_context(ckks_secret_context)

In [13]:
# Decrypt the computed radiation average (CKKS)
decrypted_average_value = encrypted_average_vector.decrypt()[0]

# Decrypt the computed enemy sum (BFV)
decrypted_sum_value = encrypted_sum_vector.decrypt()[0]

# Decrypt additional CKKS results
decrypted_radiation_score = encrypted_radiation_score_vector.decrypt()[0]
decrypted_safe_route_score = encrypted_safe_route_vector.decrypt()[0]

print(f"Decrypted radiation average: {decrypted_average_value:.4f} mSv")
print(f"Decrypted enemy sum: {decrypted_sum_value}")
print(f"Decrypted radiation score: {decrypted_radiation_score:.4f}")
print(f"Decrypted safe route score: {decrypted_safe_route_score:.4f}")

Decrypted radiation average: 77.7458 mSv
Decrypted enemy sum: 5899
Decrypted radiation score: 54.4221
Decrypted safe route score: 45.5779


------------------------------------------

## Verify Results

Compare decrypted result with expected value to verify correctness.

In [14]:
# Load expected values from generated data
with open("data/expected_values.json", "r") as json_file:
    expected_values_dict = json.load(json_file)

# Calculate expected values for additional computations
expected_average_value = expected_values_dict["expected_average_ckks"]
expected_sum_value = expected_values_dict["expected_sum_bfv"]

# Expected radiation score: average * weight_radiation (0.7)
expected_radiation_score = expected_average_value * 0.7

# Expected safe route score: 100 - radiation_score
expected_safe_route_score = 100.0 - expected_radiation_score

# Verify CKKS radiation average
absolute_difference_average = abs(decrypted_average_value - expected_average_value)
relative_error_average = (absolute_difference_average / expected_average_value) * 100

# Verify BFV enemy sum
absolute_difference_sum = abs(decrypted_sum_value - expected_sum_value)

# Verify CKKS radiation score
absolute_difference_score = abs(decrypted_radiation_score - expected_radiation_score)
relative_error_score = (absolute_difference_score / expected_radiation_score) * 100

# Verify CKKS safe route score
absolute_difference_route = abs(decrypted_safe_route_score - expected_safe_route_score)
relative_error_route = (absolute_difference_route / abs(expected_safe_route_score)) * 100

# Display CKKS verification (Radiation Average)
print(f"\n{'='*50}")
print(f"CKKS VERIFICATION (Radiation Average)")
print(f"{'='*50}")
print(f"Expected: {expected_average_value:.4f} mSv")
print(f"Decrypted: {decrypted_average_value:.4f} mSv")
print(f"Absolute difference: {absolute_difference_average:.6f} mSv")
print(f"Relative error: {relative_error_average:.4f}%")
print(f"Status: {'✅ PASSED' if relative_error_average < 1.0 else '✗ FAILED'}")

# Display BFV verification (Enemy Sum)
print(f"\n{'='*50}")
print(f"BFV VERIFICATION (Enemy Sum)")
print(f"{'='*50}")
print(f"Expected: {expected_sum_value}")
print(f"Decrypted: {int(decrypted_sum_value)}")
print(f"Difference: {absolute_difference_sum}")
print(f"Status: {'✅ PASSED' if absolute_difference_sum == 0 else '✗ FAILED'}")

# Display CKKS verification (Radiation Score)
print(f"\n{'='*50}")
print(f"CKKS VERIFICATION (Weighted Radiation Score)")
print(f"{'='*50}")
print(f"Expected: {expected_radiation_score:.4f} (average * 0.7)")
print(f"Decrypted: {decrypted_radiation_score:.4f}")
print(f"Absolute difference: {absolute_difference_score:.6f}")
print(f"Relative error: {relative_error_score:.4f}%")
print(f"Status: {'✅ PASSED' if relative_error_score < 1.0 else '✗ FAILED'}")

# Display CKKS verification (Safe Route Score)
print(f"\n{'='*50}")
print(f"CKKS VERIFICATION (Safe Route Score)")
print(f"{'='*50}")
print(f"Expected: {expected_safe_route_score:.4f} (100 - score)")
print(f"Decrypted: {decrypted_safe_route_score:.4f}")
print(f"Absolute difference: {absolute_difference_route:.6f}")
print(f"Relative error: {relative_error_route:.4f}%")
print(f"Status: {'✅ PASSED' if relative_error_route < 1.0 else '✗ FAILED'}")

print(f"\n{'='*50}")
print(f"Note: CKKS is approximate - small errors are expected")
print(f"Note: BFV is exact - no errors expected for integers")


CKKS VERIFICATION (Radiation Average)
Expected: 77.7457 mSv
Decrypted: 77.7458 mSv
Absolute difference: 0.000056 mSv
Relative error: 0.0001%
Status: ✅ PASSED

BFV VERIFICATION (Enemy Sum)
Expected: 5899
Decrypted: 5899
Difference: 0
Status: ✅ PASSED

CKKS VERIFICATION (Weighted Radiation Score)
Expected: 54.4220 (average * 0.7)
Decrypted: 54.4221
Absolute difference: 0.000075
Relative error: 0.0001%
Status: ✅ PASSED

CKKS VERIFICATION (Safe Route Score)
Expected: 45.5780 (100 - score)
Decrypted: 45.5779
Absolute difference: 0.000075
Relative error: 0.0002%
Status: ✅ PASSED

Note: CKKS is approximate - small errors are expected
Note: BFV is exact - no errors expected for integers
